# 基于物品的协同过滤
- 计算物品->用户的矩阵
- 计算物品同现矩阵
- 计算物品的协同过滤矩阵
- 计算用户浏览过物品的相近物品

In [7]:
import os

In [12]:
def get_user_click(rating_file):
    """
    get user clcik list
    Args:
        rating_file:input_file
    Return:
        dict,key:userid,value:[item1,item2]
    """
    fp = open(rating_file)
    num=0
    user_click={}
    for line in fp:
        items = line.strip().split("::")
        uid = items[0]
        mid = items[1]
        rating = items[2]
        if float(rating)<3.0:
            continue
        if uid not in user_click:
            user_click[uid]=[]
        user_click[uid].append(mid)
    return user_click

In [13]:
user_click = get_user_click("ml-1m/ratings.dat")

In [14]:
print(user_click["1"])

['1193', '661', '914', '3408', '2355', '1197', '1287', '2804', '594', '919', '595', '938', '2398', '2918', '1035', '2791', '2687', '2018', '3105', '2797', '2321', '720', '1270', '527', '2340', '48', '1097', '1721', '1545', '745', '2294', '3186', '1566', '588', '1907', '783', '1836', '1022', '2762', '150', '1', '1961', '1962', '2692', '260', '1028', '1029', '1207', '2028', '531', '3114', '608', '1246']


In [69]:
import codecs
def get_item_info(item_file):
    """
    get item info[title,genres]
    Args:
        item_file:input iteminfo file
    Return:
        a dict,key itemid,value:[title,genres]
    
    """
    fp = codecs.open(item_file,'r',encoding = "ISO-8859-1")
    item_info = {}
    for line in fp:
        items = line.strip().split("::")
        if items[0] not in item_info:
            item_info[items[0]]=[items[1],items[2]]
    return item_info        

In [71]:
item_info = get_item_info("ml-1m/movies.dat")
print(item_info["1"])
print(len(item_info))

['Toy Story (1995)', "Animation|Children's|Comedy"]
3883


In [31]:
def base_contribute_score():
    return 1

## 同现矩阵

In [37]:
co_appear={}
item_user_click_times={}
for user,itemlist in user_click.items():
    for index_i in range(0,len(itemlist)):
        itemid_i = itemlist[index_i]
        item_user_click_times.setdefault(itemid_i,0)
        item_user_click_times[itemid_i]+=1
        for index_j in range(index_i+1,len(itemlist)):
            itemid_j = itemlist[index_j]
            co_appear.setdefault(itemid_i,{})
            co_appear.setdefault(itemid_j,{})
            co_appear[itemid_i].setdefault(itemid_j,0)
            co_appear[itemid_j].setdefault(itemid_i,0)
            co_appear[itemid_i][itemid_j] += base_contribute_score()
            co_appear[itemid_j][itemid_i] += base_contribute_score()  

## 协同矩阵

In [45]:
import math
import operator
item_sim_score = {}
item_sim_score_sorted = {}
for itemid_i,relate_item in co_appear.items():
    for itemid_j,co_time in relate_item.items():
        item_sim_score.setdefault(itemid_i,{})
        item_sim_score[itemid_i].setdefault(itemid_j,0)
        item_sim_score[itemid_i][itemid_j] = co_time/math.sqrt(item_user_click_times[itemid_j]*item_user_click_times[itemid_i])
for itemid in item_sim_score:
    item_sim_score_sorted[itemid] = sorted(item_sim_score[itemid].items(),key=operator.itemgetter(1),reverse=True)

In [47]:
item_sim_score_sorted["1"]

[('3114', 0.612198986396246),
 ('1265', 0.6020552572377078),
 ('588', 0.5864099493225665),
 ('2355', 0.5636653032685426),
 ('1270', 0.5621910172151441),
 ('34', 0.5557245119336681),
 ('356', 0.5519724544797965),
 ('2571', 0.5475094878419099),
 ('1197', 0.5456439805070591),
 ('1196', 0.5453601357295754),
 ('260', 0.5417689303413047),
 ('1580', 0.5370068532804573),
 ('364', 0.5353508971603382),
 ('2396', 0.5314178019722807),
 ('1198', 0.5251892001135321),
 ('318', 0.5246436043420603),
 ('457', 0.5232761005270472),
 ('2028', 0.5227149935356792),
 ('1923', 0.5185523751963246),
 ('589', 0.5173900916860414),
 ('2716', 0.5169503737615713),
 ('593', 0.5162910469455431),
 ('1210', 0.5144454168252343),
 ('2797', 0.514134978711599),
 ('1097', 0.513079173099724),
 ('2918', 0.5119410401913249),
 ('2762', 0.5105234293426275),
 ('595', 0.5080682368868706),
 ('296', 0.5062175003251744),
 ('1136', 0.505590482794552),
 ('480', 0.5052767559297423),
 ('608', 0.5010072099061046),
 ('2858', 0.50075635722925

In [52]:
## 取相识计算
def cal_recom_result(sim_info,user_click):
    recent_click_num = 3
    topk=5
    recom_info = {}
    for user in user_click:
        recom_info.setdefault(user,{})
        click_list = user_click[user]
        for itemid in click_list[:recent_click_num]:
            for itemsimzuhe in sim_info[itemid][:topk]:
                itemsimid = itemsimzuhe[0]
                itemsimscore = itemsimzuhe[1]
                recom_info[user][itemsimid]= itemsimscore
    return recom_info

In [53]:
recom_result = cal_recom_result(item_sim_score_sorted,user_click)

In [54]:
recom_result["1"]

{'1022': 0.37946312147705386,
 '1028': 0.49163471985912854,
 '1029': 0.37439179041757437,
 '1035': 0.4952254020433806,
 '1225': 0.5118635212557388,
 '1247': 0.5148592176673679,
 '1947': 0.5832761273205334,
 '2565': 0.5060233745526537,
 '318': 0.5198255105750703,
 '364': 0.39249698357306506,
 '551': 0.40776389576686833,
 '595': 0.3971570575568227,
 '608': 0.5255996536174563,
 '858': 0.5324877380347874,
 '899': 0.525820107323753}

In [74]:
print(item_info['1947'])
print(item_info['1'])
print(item_info['899'])

['West Side Story (1961)', 'Musical|Romance']
['Toy Story (1995)', "Animation|Children's|Comedy"]
["Singin' in the Rain (1952)", 'Musical|Romance']


In [68]:
print(len(item_info))

1082
